In [ ]:
# Uncomment and run the appropriate command for your operating system, if required
# No installation is reqiured on Google Colab / Kaggle notebooks

# Linux / Binder / Windows (No GPU)
# !pip install numpy matplotlib torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Linux / Windows (GPU)
# pip install numpy matplotlib torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
 
# MacOS (NO GPU)
# !pip install numpy matplotlib torch torchvision torchaudio

# Installing necessary packages

In [1]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

required_packages = ['numpy', 'matplotlib', 'torch', 'torchvision', 'torchaudio', 'transformers', 'pillow', 'json']

for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

In [3]:
import os
import torch
import json
import torch.nn as nn
import re
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
from PIL import Image, ImageFilter, ImageEnhance
from transformers import BertTokenizer, BertModel
import shutil
import matplotlib.pyplot as plt

## creating a local tokenizer

In [4]:
def create_tokenizer(text_dir):
    # Extract unique keywords
    unique_words = set()
    text_files = [f for f in os.listdir(text_dir) if f.endswith('.txt')]
    for file_name in text_files:
        file_path = os.path.join(text_dir, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read().lower()
            words = re.split(r',\s*', content)
            unique_words.update(words)
    
    # Create a mapping from keywords to indices
    word_to_idx = {word: idx for idx, word in enumerate(sorted(unique_words))}
    
    # Save the tokenizer for later use
    with open('tokenizer.json', 'w') as f:
        json.dump(word_to_idx, f)
    
    return word_to_idx

word_to_idx = create_tokenizer('dataset/text')
print(f"Tokenized keywords: {word_to_idx}")

Tokenized keywords: {'abstract': 0, 'abstract swirls': 1, 'africa map': 2, 'avif': 3, 'back': 4, 'big head': 5, 'birds': 6, 'black': 7, 'black and white': 8, 'blob': 9, 'blue': 10, 'bluee': 11, 'blues': 12, 'boy': 13, 'browm': 14, 'brown': 15, 'circles': 16, 'colorful': 17, 'colorfull': 18, 'colorufl': 19, 'community gathering': 20, 'contemporary': 21, 'cool': 22, 'cubism': 23, 'cultural': 24, 'dance': 25, 'dark': 26, 'detailed': 27, 'dull': 28, 'egyptian': 29, 'elephant': 30, 'environmental': 31, 'expressionism': 32, 'face': 33, 'family': 34, 'feminism': 35, 'feminist': 36, 'festival': 37, 'figurative': 38, 'flat': 39, 'flora': 40, 'folk': 41, 'geometric': 42, 'geometric background': 43, 'girl': 44, 'grass': 45, 'green': 46, 'group': 47, 'group gathering': 48, 'group portrait': 49, 'hands': 50, 'house': 51, 'individual': 52, 'indiviual': 53, 'landscape': 54, 'line art': 55, 'lines': 56, 'man': 57, 'market': 58, 'mask': 59, 'men': 60, 'mess': 61, 'minimalist': 62, 'modern': 63, 'monoch

# Data Preprocessing

In [ ]:
base_dir = 'dataset'
image_folders = ['nomadic', 'cubism', 'print', 'gray']
output_image_dir = os.path.join(base_dir, 'images')
output_text_dir = os.path.join(base_dir, 'text')

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_text_dir, exist_ok=True)

# Define the list of filters and enhancements
filters = [
    ('BLUR', ImageFilter.BLUR),
    ('CONTOUR', ImageFilter.CONTOUR),
    ('DETAIL', ImageFilter.DETAIL),
    ('EDGE_ENHANCE', ImageFilter.EDGE_ENHANCE),
    ('EDGE_ENHANCE_MORE', ImageFilter.EDGE_ENHANCE_MORE),
    ('EMBOSS', ImageFilter.EMBOSS),
    ('FIND_EDGES', ImageFilter.FIND_EDGES),
    ('SHARPEN', ImageFilter.SHARPEN),
    ('SMOOTH', ImageFilter.SMOOTH),
    ('SMOOTH_MORE', ImageFilter.SMOOTH_MORE)
]

enhancements = [
    ('BRIGHTNESS', ImageEnhance.Brightness, 1.5),  # Increase brightness by 1.5
    ('CONTRAST', ImageEnhance.Contrast, 1.5),  # Increase contrast by 1.5
    ('COLOR', ImageEnhance.Color, 1.5),  # Increase color by 1.5
    ('SHARPNESS', ImageEnhance.Sharpness, 2.0)  # Increase sharpness by 2.0
]

additional_filters = [
    ('GAUSSIAN_BLUR', ImageFilter.GaussianBlur(radius=2)),
    ('UNSHARP_MASK', ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3)),
    ('MEDIAN_FILTER', ImageFilter.MedianFilter(size=3)),
    ('MIN_FILTER', ImageFilter.MinFilter(size=3)),
    ('MAX_FILTER', ImageFilter.MaxFilter(size=3)),
    ('MODE_FILTER', ImageFilter.ModeFilter(size=3))
]

all_operations = filters + enhancements + additional_filters

# Function to apply filters and enhancements
def apply_operations(image, output_dir, base_filename, counter):
    original_image_path = os.path.join(output_dir, f"{base_filename}{counter:03d}.png")
    image.save(original_image_path)
    counter += 1

    for operation_name, operation in filters:
        modified_image = image.filter(operation)
        modified_image.save(os.path.join(output_dir, f"{base_filename}{counter:03d}_{operation_name}.png"))
        counter += 1

    for operation_name, operation, factor in enhancements:
        enhancer = operation(image)
        modified_image = enhancer.enhance(factor)
        modified_image.save(os.path.join(output_dir, f"{base_filename}{counter:03d}_{operation_name}.png"))
        counter += 1

    for operation_name, operation in additional_filters:
        modified_image = image.filter(operation)
        modified_image.save(os.path.join(output_dir, f"{base_filename}{counter:03d}_{operation_name}.png"))
        counter += 1

    return counter

# Initialize the counter
image_counter = 1

# Process each folder
for folder in image_folders:
    current_folder_path = os.path.join(base_dir, 'images', folder)
    for filename in os.listdir(current_folder_path):
        if filename.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
            image_path = os.path.join(current_folder_path, filename)
            image = Image.open(image_path)
            base_filename = "artwork"

            # Apply operations and save images
            image_counter = apply_operations(image, output_image_dir, base_filename, image_counter)

            # Create corresponding text file
            for i in range(image_counter - 21, image_counter):
                txt_filename = f"{base_filename}{i:03d}.txt"
                with open(os.path.join(output_text_dir, txt_filename), 'w') as f:
                    f.write(folder)

print("Processing complete.")

In [5]:
from PIL import UnidentifiedImageError

image_dir = 'dataset/images'
files = os.listdir(image_dir)  # list of all files in the directory

counter = 1

# pattern to match files already in the `artworkXXX.ext` format
pattern = re.compile(r'artwork\d{3}\.\w+')

for file_name in files:
    file_ext = os.path.splitext(file_name)[1]
    
    old_file_path = os.path.join(image_dir, file_name)
    new_file_path = old_file_path
    
    if not pattern.match(file_name):
        new_file_name = f'artwork{counter:03d}{file_ext}'
        new_file_path = os.path.join(image_dir, new_file_name)
        os.rename(old_file_path, new_file_path)
        counter += 1
    
    try:
        # Open the image
        with Image.open(new_file_path) as img:
            # Check if the image size is not 128x128
            if img.size != (128, 128):
                img_resized = img.resize((128, 128))
                img_resized.save(new_file_path)
    except (UnidentifiedImageError, OSError) as e:
        print(f"Skipping invalid file: {file_name}, error: {e}")

print("Renaming and resizing completed.")

Renaming and resizing completed.


In [6]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import json

class TextImageDataset(Dataset):
    def __init__(self, text_dir, image_dir, transform=None, tokenizer_path='tokenizer.json'):
        self.text_dir = text_dir
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))
        self.text_files = sorted([f for f in os.listdir(text_dir) if f.endswith('.txt')])
        
        # Load the tokenizer
        with open(tokenizer_path, 'r') as f:
            self.word_to_idx = json.load(f)
        
        self.valid_image_files = self.filter_valid_images()
        
        if len(self.text_files) != len(self.valid_image_files):
            raise ValueError("Number of text files does not match number of valid image files.")

    def filter_valid_images(self):
        valid_files = []
        for img_name in self.image_files:
            img_path = os.path.join(self.image_dir, img_name)
            try:
                img = Image.open(img_path)
                img.verify()  # Verify that it is, in fact, an image
                valid_files.append(img_name)
            except (UnidentifiedImageError, IOError):
                print(f"Invalid image file: {img_path}")
        return valid_files

    def __len__(self):
        return len(self.valid_image_files)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.valid_image_files[idx])
        image = Image.open(img_name)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        txt_name = os.path.join(self.text_dir, self.text_files[idx])
        description = self.read_text_file(txt_name)
        
        tokens = self.tokenize_description(description)
        return image, tokens

    def read_text_file(self, file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        except UnicodeDecodeError:
            with open(file_path, 'r', encoding='latin1') as f:
                return f.read()
    
    def tokenize_description(self, description):
        words = re.split(r',\s*', description.lower())
        tokens = [self.word_to_idx[word] for word in words if word in self.word_to_idx]
        return torch.tensor(tokens, dtype=torch.long)

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

dataset = TextImageDataset(text_dir='dataset/text', image_dir='dataset/images', transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

print("Dataset and DataLoader created successfully.")

Dataset and DataLoader created successfully.


# Feature Engineering
Defining the Generator and Discriminator networks

In [8]:
class Generator(nn.Module):
    def __init__(self, latent_dim, text_dim, img_channels):
        super(Generator, self).__init__()
        self.text_embedding = nn.Embedding(text_dim, latent_dim)
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(latent_dim * 2, 512, 4, 1, 0),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, img_channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, noise, text):
        text_embedding = self.text_embedding(text).sum(dim=1)
        x = torch.cat([noise, text_embedding], dim=1)
        x = x.unsqueeze(2).unsqueeze(3)
        return self.gen(x)

class Discriminator(nn.Module):
    def __init__(self, img_channels, text_dim, hidden_dim, img_size=128):
        super(Discriminator, self).__init__()

        def conv_output_size(size, kernel_size=4, stride=2, padding=1):
            return (size - kernel_size + 2 * padding) // stride + 1

        self.img_size = img_size
        self.hidden_dim = hidden_dim

        current_size = img_size
        self.img_dis = nn.Sequential(
            nn.Conv2d(img_channels, hidden_dim // 4, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )
        current_size = conv_output_size(current_size)

        self.img_dis.add_module("conv2", nn.Conv2d(hidden_dim // 4, hidden_dim // 2, 4, stride=2, padding=1))
        self.img_dis.add_module("lrelu2", nn.LeakyReLU(0.2, inplace=True))
        current_size = conv_output_size(current_size)

        self.img_dis.add_module("conv3", nn.Conv2d(hidden_dim // 2, hidden_dim, 4, stride=2, padding=1))
        self.img_dis.add_module("lrelu3", nn.LeakyReLU(0.2, inplace=True))
        current_size = conv_output_size(current_size)

        self.flattened_img_size = hidden_dim * (current_size * current_size)

        self.text_embedding = nn.Embedding(text_dim, hidden_dim * (current_size * current_size))
        self.final = nn.Linear(self.flattened_img_size + self.flattened_img_size, 1)

    def forward(self, img, text):
        img_out = self.img_dis(img).view(img.size(0), -1)
        text_embedding = self.text_embedding(text).sum(dim=1)
        x = torch.cat([img_out, text_embedding], dim=1)
        return self.final(x)

text_dim = len(word_to_idx)  # Set the text dimension to the number of unique words
latent_dim = 100
hidden_dim = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = Generator(latent_dim, text_dim, 3).to(device)
discriminator = Discriminator(3, text_dim, hidden_dim, img_size=128).to(device)

# Training the Model
The generator and discriminator are trained here using a loop.
## Optimization
The optimization is embedded within the training loop. The code uses Adam optimizer to update the weights of both the generator and the discriminator.
## Evaluation
Evaluation happens within the training loop where loss values for both the generator and the discriminator are printed every 100 steps. Additionally, generated sample images are saved at the end of each epoch to visualize the generator’s progress.

In [9]:
import torch.optim as optim
import torch.nn as nn
import os

def train_gan(generator, discriminator, dataloader, epochs, lr, device):
    criterion = nn.BCEWithLogitsLoss()
    optim_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optim_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    for epoch in range(epochs):
        for i, (images, text_ids) in enumerate(dataloader):
            batch_size = images.size(0)
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            images = images.to(device)
            text_ids = text_ids.to(device)

            optim_d.zero_grad()
            outputs = discriminator(images, text_ids)
            real_loss = criterion(outputs, real_labels)
            real_loss.backward()

            noise = torch.randn(batch_size, latent_dim).to(device)
            fake_images = generator(noise, text_ids)
            outputs = discriminator(fake_images.detach(), text_ids)
            fake_loss = criterion(outputs, fake_labels)
            fake_loss.backward()
            optim_d.step()

            optim_g.zero_grad()
            outputs = discriminator(fake_images, text_ids)
            g_loss = criterion(outputs, real_labels)
            g_loss.backward()
            optim_g.step()

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(dataloader)}], '
                    f'D Loss: {real_loss.item() + fake_loss.item():.4f}, G Loss: {g_loss.item():.4f}')

        with torch.no_grad():
            sample_noise = torch.randn(batch_size, latent_dim).to(device)
            sample_images = generator(sample_noise, text_ids)
            os.makedirs('samples', exist_ok=True)
            utils.save_image(sample_images, f'samples/sample_epoch_{epoch + 1}.png', nrow=8, normalize=True)

torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')            